Install requirements

In [2]:
!python3 -m pip install -q --upgrade pip
!python3 -m pip install -q lancedb
!python3 -m pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/f5/d0/2e455d894ec0d6527e662ad55e70c04f421ad83a6fd0a54c3dd73c411282/opencv_python-4.8.0.76-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 11.9 MB/s eta 0:00:0000:0100:01


In [1]:
import lancedb

uri = r'../data/test.lancedb'
db = lancedb.connect(uri)

In [2]:
from dataclasses import dataclass
import cv2

@dataclass
class BoundingBox:
    x: int
    y: int
    width: int
    height: int

@dataclass
class Annotation:
    className: str
    bbox: BoundingBox

@dataclass
class AnnotatedImage:
    image: cv2.Mat
    annotations: list[Annotation]

In [16]:
from xml.etree import ElementTree
from pathlib import Path

def load_annotations(dataset_path: Path):
    for file in dataset_path.iterdir():
        if not file.name.endswith('.xml'):
            continue

        parsed_annotation = ElementTree.parse(file)
        image_internal_path = parsed_annotation.find('path')

        image_path = dataset_path.joinpath(image_internal_path.text)
        image = cv2.imread(str(image_path))

        objects = parsed_annotation.findall('object')

        annotations: list[Annotation] = []

        for o in objects:
            object_name = o.find('name').text
            object_bbox = o.find('bndbox')

            bbox_xmin = round(float(object_bbox.find('xmin').text))
            bbox_ymin = round(float(object_bbox.find('ymin').text))
            bbox_xmax = round(float(object_bbox.find('xmax').text))
            bbox_ymax = round(float(object_bbox.find('ymax').text))

            bounding_box = BoundingBox(bbox_xmin, bbox_ymin, bbox_xmax - bbox_xmin, bbox_ymax - bbox_ymin)
            annotation = Annotation(object_name, bounding_box)

            annotations.append(annotation)

        annotatedImage = AnnotatedImage(image, annotations)

        print(annotatedImage)

AnnotatedImage(image=array([[[ 15,  26,  40],
        [ 15,  26,  40],
        [ 15,  26,  40],
        ...,
        [138, 152, 164],
        [125, 139, 151],
        [115, 129, 141]],

       [[ 13,  24,  38],
        [ 14,  25,  39],
        [ 14,  25,  39],
        ...,
        [131, 145, 157],
        [129, 143, 155],
        [127, 141, 153]],

       [[ 13,  24,  38],
        [ 13,  24,  38],
        [ 13,  24,  38],
        ...,
        [154, 168, 180],
        [156, 170, 182],
        [159, 173, 185]],

       ...,

       [[134, 131, 133],
        [133, 130, 132],
        [132, 129, 131],
        ...,
        [116, 131, 140],
        [116, 131, 140],
        [116, 131, 140]],

       [[133, 130, 132],
        [130, 127, 129],
        [127, 124, 126],
        ...,
        [114, 129, 138],
        [113, 128, 137],
        [112, 127, 136]],

       [[131, 128, 130],
        [127, 124, 126],
        [124, 121, 123],
        ...,
        [110, 125, 134],
        [108, 123, 132],
   

AttributeError: 'NoneType' object has no attribute 'find'